In [1]:
from pynq import Overlay
from time import sleep
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
ol = Overlay('/home/xilinx/pynq/overlays/RO/RO.bit')

In [3]:
ol?

The register addresses are: 0X00, 0X04, 0X08, 0X0C

In [13]:
ol.AXI4_heater_0.read(0x0000 + 0x0004)

0

In [4]:
import numpy

def smooth(x,window_len=11,window='hanning'):
    """smooth the data using a window with requested size.
    
    This method is based on the convolution of a scaled window with the signal.
    The signal is prepared by introducing reflected copies of the signal 
    (with the window size) in both ends so that transient parts are minimized
    in the begining and end part of the output signal.
    
    input:
        x: the input signal 
        window_len: the dimension of the smoothing window; should be an odd integer
        window: the type of window from 'flat', 'hanning', 'hamming', 'bartlett', 'blackman'
            flat window will produce a moving average smoothing.

    output:
        the smoothed signal
    
    see also: 
    
    numpy.hanning, numpy.hamming, numpy.bartlett, numpy.blackman, numpy.convolve
    """

    if window_len<3:
        return x


   # if not window in ['flat', 'hanning', 'hamming', 'bartlett', 'blackman']:
   #     raise ValueError, "Window is on of 'flat', 'hanning', 'hamming', 'bartlett', 'blackman'"


    s=numpy.r_[x[window_len-1:0:-1],x,x[-2:-window_len-1:-1]]
    #print(len(s))
    if window == 'flat': #moving average
        w=numpy.ones(window_len,'d')
    else:
        w=eval('numpy.'+window+'(window_len)')

    y=numpy.convolve(w/w.sum(),s,mode='valid')
    return y

In [5]:
def XADC_temp():
    return ((ol.Temp_sensor.read(0x200) >> 4) * 503.975/4096 - 273.15)

In [ ]:
%%time
sensor_1 = []
sensor_2 = []
sensor_3 = []
sensor_4 = []
temperature = []
heater_reg_address = 0x00
for i in range(1000):
    if i > 0 and i < 100:
        for j in range(16):
            ol.AXI4_heater_0.write(heater_reg_address,0xFFFFFFFF)#(0x00,0x000003FF)
            heater_reg_address = heater_reg_address + 0x04
        heater_reg_address = 0x00
    else:
        for j in range(16):
            ol.AXI4_heater_0.write(heater_reg_address,0x00000000)#(0x00,0x000003FF)
            heater_reg_address = heater_reg_address + 0x04
        heater_reg_address = 0x00
    sensor_1.append(ol.Frequency_counter_AX_0.read(0x00))
    sensor_2.append(ol.Frequency_counter_AX_0.read(0x04))
    sensor_3.append(ol.Frequency_counter_AX_0.read(0x08))
    sensor_4.append(ol.Frequency_counter_AX_0.read(0x0C))
    temperature.append(XADC_temp())
    sleep(0.01)

In [ ]:
window_len = 30
#fig = plt.figure(figsize=(10,6))
plt.plot(smooth([i / 1000 for i in sensor_1],window_len,window='hanning'))
#plt.plot([i / 1000 for i in sensor_2])
#plt.plot([i / 1000 for i in sensor_3])
#plt.plot([i / 1000 for i in sensor_4])
plt.ylabel('MHz')
plt.xlabel('Sample Number')

In [ ]:

plt.plot(smooth(temperature,window_len,window='hanning'))

In [ ]:
N = 50
plt.plot(np.convolve(temperature, np.ones(N)/N, mode='valid'))

In [ ]:
plt.hist(sensor_4,70)

In [11]:
ol = Overlay('/home/xilinx/pynq/overlays/base/base.bit')